## Base model with default params

In [52]:
train = pd.read_csv('recommendationsv4/train.csv')
# delete duplicates
train.drop_duplicates(inplace=True)
# build user-item interactions matrix
ratings_coo = sparse.coo_matrix(
    (train['rating'].astype(int), (train['userid'], train['itemid'])))

C:\ProgramData\Anaconda33\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [53]:
start_time = time.time()
NUM_THREADS = 2  # number of threads
NUM_COMPONENTS = 20  # number of vector components
NUM_EPOCHS = 10  # number of learning epochs

# split ratings to train and test
train_rating_coo, test_rating_coo = random_train_test_split(ratings_coo)

# define model and train
model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(train_rating_coo, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

# make predictions with test
preds = model.predict(test_rating_coo.row, test_rating_coo.col)
print("total time:", round(time.time() - start_time, 2), "sec.")

In [54]:
'''
# calculating metrics (ROC-AUC, precision_at_k, recall_at_k) and measuring execution time
start_time = time.time()
print(
    f'ROC-AUC (sklearn) = {sklearn.metrics.roc_auc_score(test_rating_coo.data,preds)}')
print("total time:", round(time.time() - start_time, 2), "sec.")
start_time = time.time()
print(
    f'ROC-AUC (LFM) = {auc_score(model=model, test_interactions=test_rating_coo).mean()}')
print("total time:", round(time.time() - start_time, 2), "sec.")
start_time = time.time()
print(
    f'Precision_at_k (LFM) = {precision_at_k(model=model, test_interactions=test_rating_coo, k=5).mean()}')
print("total time:", round(time.time() - start_time, 2), "sec.")
start_time = time.time()
print(
    f'Recall_at_k (LFM) = {recall_at_k(model=model, test_interactions=test_rating_coo, k=5).mean()}')
print("total time:", round(time.time() - start_time, 2), "sec.")
'''

'\n# calculating metrics (ROC-AUC, precision_at_k, recall_at_k) and measuring execution time\nstart_time = time.time()\nprint(\n    f\'ROC-AUC (sklearn) = {sklearn.metrics.roc_auc_score(test_rating_coo.data,preds)}\')\nprint("total time:", round(time.time() - start_time, 2), "sec.")\nstart_time = time.time()\nprint(\n    f\'ROC-AUC (LFM) = {auc_score(model=model, test_interactions=test_rating_coo).mean()}\')\nprint("total time:", round(time.time() - start_time, 2), "sec.")\nstart_time = time.time()\nprint(\n    f\'Precision_at_k (LFM) = {precision_at_k(model=model, test_interactions=test_rating_coo, k=5).mean()}\')\nprint("total time:", round(time.time() - start_time, 2), "sec.")\nstart_time = time.time()\nprint(\n    f\'Recall_at_k (LFM) = {recall_at_k(model=model, test_interactions=test_rating_coo, k=5).mean()}\')\nprint("total time:", round(time.time() - start_time, 2), "sec.")\n'

### Результаты 

* model fit-predict = (5 sec.)
* ROC-AUC (sklearn) = 0.72814 (0.07 sec.)
* ROC-AUC (LFM) = 0.71985 (192 sec.)
* Precision_at_k (LFM) = 0.01306 (176 sec.)
* Recall_at_k (LFM) = 0.04045 (196 sec.)

Можно видеть, что метрики ROC-AUC практически одинаковы для обеих библиотек, однако время подсчёта отличается 
на порядки. В среднем, вычисление всех трех метрик встроенными функциями библиотеки LightFm происходит за время
около трёх минут каждая, что почти в 40 раз превышает время обучения модели и предсказания. В дальнейшем при подборе гиперпараметров для экономии времени будем вычислять только 
ROC-AUC, а другие метрики - только лишь для финальной модели. 

## Hyperparameters

Попробуем теперь улучшить метрики за счёт подбора гиперпараметров. Используем самописный перебор по сетке параметров. 

Варьировать будем параметры: 
* loss_types = {"logistic", "warp"}
* n_components = {20, 25, 30, 35}
* n_epochs = {5, 10, 15, 20}

Параметр, отвечающий за число кросс-валидаций выберем равным 3. Затем выберем оптимальную модель из условия наилучшего среднего значения усредненной метрики ROC-AUC. 

In [55]:
def solve(n_treads, n_components, n_epochs, loss_type, train_rating_coo, test_rating_coo):
    """function returns ROC_AUC score for current model"""
    model = LightFM(learning_rate=0.1, loss=loss_type,
                    no_components=n_components)
    model = model.fit(train_rating_coo, epochs=n_epochs, num_threads=n_treads)
    preds = model.predict(test_rating_coo.row, test_rating_coo.col)
    return sklearn.metrics.roc_auc_score(test_rating_coo.data, preds)


def solve_metrics(n_treads, n_components, n_epochs, loss_type, train_rating_coo, test_rating_coo):
    """function returns precision and recall at_k score for current model"""
    model = LightFM(learning_rate=0.1, loss=loss_type,
                    no_components=n_components)
    model = model.fit(train_rating_coo, epochs=n_epochs, num_threads=n_treads)
    start_time = time.time()
    pr_at_k = precision_at_k(
        model=model, test_interactions=test_rating_coo, k=5).mean()
    print(f'Precision_at_k (LFM) = {pr_at_k}')
    print("total time:", round(time.time() - start_time, 2), "sec.")
    start_time = time.time()
    rc_at_k = recall_at_k(
        model=model, test_interactions=test_rating_coo, k=5).mean()
    print(f'Recall_at_k (LFM) = {rc_at_k}')
    print("total time:", round(time.time() - start_time, 2), "sec.")
    return pr_at_k, rc_at_k


def submission_best(loss_type, components, epochs_val, threads):
    model = LightFM(learning_rate=0.1, loss=loss_type,
                    no_components=components)
    model = model.fit(ratings_coo, epochs=epochs_val, num_threads=threads)
    preds = model.predict(test.userid.values, test.itemid.values)
    preds = (preds - preds.min())/(preds.max() - preds.min())
    submission['rating'] = preds
    submission.to_csv('submission_base.csv', index=False)
    print("Submission successifully wrote in file")


def write_model_embeddings(loss_type, components, epochs_val, threads):
    """function writes model and embeddings in files"""
    model = LightFM(learning_rate=0.1, loss=loss_type,
                    no_components=components)
    model = model.fit(ratings_coo, epochs=epochs_val, num_threads=threads)
    item_biases, item_embeddings = model.get_item_representations()
    user_biases, user_embeddings = model.get_user_representations()
    names = ["item_embeddings", "user_embeddings", "model"]
    data = [item_embeddings, user_embeddings, model]
    for idx, name in enumerate(names):
        with open(name + '.pkl', 'wb') as file:
            pickle.dump(data[idx], file)
            file.close()

In [56]:
'''
loss_types = ["logistic", "warp"]
n_treads = 8
n_components = [i for i in range(74, 75, 1)]
n_epochs = [i for i in range(14, 15, 1)]
n_cv = 3
result = pd.DataFrame(
    columns=['n_comp', 'n_epochs', 'loss_type', 'roc_auc', 'aver_roc_auc'])
idx = 0
all_iters = len(n_components) * len(n_epochs) * len(loss_types)
start_time = time.time()
for components in n_components:
    for epochs in n_epochs:
        for loss in loss_types:
            roc_auc = []
            models = []
            for _ in range(n_cv):
                train_rating_coo, test_rating_coo = random_train_test_split(
                    ratings_coo)
                answer = solve(n_treads, components, epochs, loss,
                               train_rating_coo, test_rating_coo)
                roc_auc.append(round(answer, 5))
            aver_roc_auc = round(sum(roc_auc) / len(roc_auc), 4)
            idx = idx + 1
            new = pd.DataFrame(
                [[components, epochs, loss, roc_auc, aver_roc_auc]], columns=result.columns)
            result = pd.concat([result, new])
            print(
                f"Progress: {idx}/{all_iters} ({round(100 * idx / all_iters,1)}%)", end='\r')
end_time = time.time()
result.index = [i + 1 for i in range(idx)]
result.sort_values(by=['aver_roc_auc'], ascending=False, inplace=True)
result.to_csv('result_cv.csv')
best = result.iloc[0].values
best_comp = best[0]
best_epochs = best[1]
best_loss = best[2]
print("Cross-validation best params:")
print(f"n_components: {best_comp}")
print(f"n_epochs: {best_epochs}")
print(f"loss_type: {best_loss}")
print("total time:", round(end_time - start_time, 2), "sec.")
'''

Cross-validation best params:
n_components: 74
n_epochs: 14
loss_type: logistic
total time: 732.35 sec.


Из параметров перебора оптимальными стали:
* loss_types = "logistic"
* n_components = 74
* n_epochs = 14

Усредненое значение метрики ROC-AUC = 0.7291, что является улучшением результата, полученного с базовыми параметрами. 

Подсчитаем среднее значение метрик precision and recall at_k для полученных параметров:

In [ ]:
prec_at_k = []
rec_at_k = []
for _ in range(n_cv):
    train_rating_coo, test_rating_coo = random_train_test_split(ratings_coo)
    answer = solve_metrics(n_treads, best_comp, best_epochs,
                           best_loss, train_rating_coo, test_rating_coo)
    prec_at_k.append(round(answer[0], 5))
    rec_at_k.append(round(answer[1], 5))
aver_pr_at_k = round(sum(prec_at_k) / len(prec_at_k), 4)
aver_rc_at_k = round(sum(rec_at_k) / len(rec_at_k), 4)


Precision_at_k (LFM) = 0.016070792451500893
total time: 761.89 sec.
Recall_at_k (LFM) = 0.04852670181704106
total time: 760.52 sec.
Precision_at_k (LFM) = 0.0159852784126997
total time: 762.8 sec.


### Результаты

* Precision_at_k (LFM) = 0.0165
* Recall_at_k (LFM) = 0.0500

Можно видеть, что значение метрик precision и recall at_k также улучшились по сравнению с базовыми параметрами 

### Выводы

построена базовая модель, основанная на матрице взаимодействий user-item. Проведен подбор гиперпараметров,который показал улучшение значений метрик ROC-AUC, precision_at_k, recall_at_k. Значение метрики ROC-AUC, полученное в соревновании kaggle равно 0.76859. 

In [ ]:

# make submission with optimal parametres
submission_best(best_loss, best_comp, best_epochs, n_treads)


## Prototype prepare


Запишем файлы модели и представления user и item в файлы:

In [ ]:
write_model_embeddings("logistic", 75, 15, 8)

Загрузим данные о товарах:

Сформируем датафрейм items, куда для каждого item запишем общее число отзывов (Reviews), asin (Amazon Standard Identification Number) и среднюю оценку (aver_mark):

In [ ]:
meta = []
with open('recommendationsv4/meta_Grocery_and_Gourmet_Food.json', 'r') as file:
    for line in file:
        meta.append(json.loads(line))
file.close()

In [ ]:
items = train[['itemid', 'unixReviewTime']].groupby('itemid').count()
items['asin'] = train[['itemid', 'asin']].groupby('itemid').first()
items.columns = ['Reviews', 'asin']
items['aver_mark'] = train[['itemid', 'overall']].groupby('itemid').mean()
items['item_id'] = items.index

Добавим также имя для каждого item, соединив items и meta по полю asin и сохраним в файл: 

In [ ]:
meta = pd.DataFrame(meta)[["title", "asin"]]
meta.drop_duplicates(subset=['asin'], keep='last', inplace=True)
items = items.merge(meta, how='left')
items.to_csv('items.csv')

Построим и сохраним в файлы графы для быстрого поиска 

In [ ]:
names = ["item_embeddings", "user_embeddings"]
out_names = ["item_graph", "user_graph"]
for idx, name in enumerate(names):
    with open(name + '.pkl', 'rb') as file:
        embeddings = pickle.load(file)
        file.close()
    search_graph_item = nmslib.init(method='hnsw', space='cosinesimil')
    search_graph_item.addDataPointBatch(embeddings)
    search_graph_item.createIndex()
    search_graph_item.saveIndex(out_names[idx] + ".hnsw")

Напишем пробный вариант поиска рекомендаций для item и user, из которого затем создадим прототип в heroku:

In [ ]:
def get_titles(items_list, items):
    result = pd.DataFrame(items[items.item_id.isin(items_list)].title)
    result.index = [i + 1 for i in range(len(items_list))]
    return result


def nearest(id, embeddings, graph, n=5):
    # n+1 because id includes in nearest list too
    return graph.knnQuery(embeddings[id], k=n+1)[0][1:]


def load_embeddings(name):
    # load embeddings
    with open(name, 'rb') as file:
        embeddings = pickle.load(file)
        file.close()
    return embeddings


def load_items():
    # load items
    return pd.read_csv('items.csv')


item_embeddings = load_embeddings('item_embeddings.pkl')
items = load_items()
search_graph_item = nmslib.init()
search_graph_item.loadIndex("item_graph.hnsw")

n_recommends = 10

# find nearest items:
item_values = list(items.item_id)
input_str = "100"
item_for_search = int(input_str)
if item_for_search in item_values:
    title = get_titles([item_for_search], items)
    print("Item title:")
    print(title.title)
    nearest_item = nearest(int(item_for_search),
                           item_embeddings, search_graph_item, n_recommends)
    answer = get_titles(nearest_item, items)
    print("Recommendations:")
    print(answer)
else:
    print("Sorry, item_id =" + " {" + input_str + "} " + "not found")

In [ ]:
def find_best_items(user_id):
    best_items = np.array([])
    if user_id in exist_users:
        # only good marks (4, 5)
        temp = train[(train['userid'] == user_id) & (
            train['overall'].isin([5.0, 4.0]))]
        best_items = temp.sort_values(by=['overall'], ascending=False)[
            'itemid'].values[:best_n]
        # if we need more (did not find all of <best_n> items for current user)
        if len(best_items) < best_n:
            add = best_n - len(best_items)
            # find nearest users for current
            near_users = nearest(user_id, user_embeddings, search_graph_user)
            for user in near_users:
                temp = train[(train['userid'] == user) & (
                    train['overall'].isin([5.0, 4.0]))]
                temp_best = temp.sort_values(by=['overall'], ascending=False)[
                    'itemid'].values[:add]
                best_items = np.concatenate((best_items, temp_best))
                add = best_n - len(best_items)
                if len(best_items) == best_n:
                    return best_items
    return best_items


def find_recommend_items(best_items):
    recommend_items = set()
    list_items = []
    # if found any best items for user, build a recommend list
    if (len(best_items) <= best_n) and (len(best_items) > 0):
        for item in best_items:
            recommend_items.update(
                nearest(item, item_embeddings, search_graph_item))
            list_items = list(recommend_items)
            if len(list_items) >= n_recommends:
                list_items = list_items[0:n_recommends]
                break
    # if we need more recommendations, take items with good aver
    # marks and lot of reviews (popular)
    if len(list_items) != n_recommends:
        most_popular_items = items[(items.aver_mark > 4.5)
                                   & (items.Reviews > items['Reviews'].quantile(0.99))]
    # add random recommendation in recommendation list from popular
    while len(list_items) < n_recommends:
        pop_id = most_popular_items.sample(10).item_id.values
        for idx in pop_id:
            if idx not in list_items:
                list_items.append(idx)

    return list_items[0:n_recommends]


def find_popular_items():
    list_items = []
    most_popular_items = items[(items.aver_mark > 4.5)
                               & (items.Reviews > items['Reviews'].quantile(0.99))]
    # add random recommendation in recommendation list from popular
    while len(list_items) < n_recommends:
        pop_id = most_popular_items.sample(n_recommends).item_id.values
        for idx in pop_id:
            if idx not in list_items:
                list_items.append(idx)
    return list_items[0:n_recommends]


best_n = 3
item_embeddings = load_embeddings('user_embeddings.pkl')
items = load_items()
search_graph_user = nmslib.init()
search_graph_user.loadIndex("user_graph.hnsw")

exist_users = train['userid'].unique()
input_str = "101"
user_for_recommend = int(input_str)

if user_for_recommend in exist_users:
    best_items = find_best_items(user_for_recommend)
    list_items = find_recommend_items(best_items)
else:
    list_items = find_popular_items()
answer = get_titles(list_items, items)
print("Recommendations: ")
print(answer)

### Выводы

написан код, дающий рекомендации как к item_id, так и к user_id. Построен прототип с использованием сервиса heroku.